<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Modelo de lenguaje con tokenización por caracteres

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torchsummary import summary

import gradio as gr

import re
import nltk
nltk.download('punkt')
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nicolas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Datos

El dataset seleccionado es un libro en ingles llamado "Odyssey of the Dragonlords". Este libro es una aventura prediseñada para Dungeons Masters y la version 5.0 de Dungeons & Dragons. El documento original está en formato .pdf y se utilizaron herramientas online para transformarlo a .txt.

A grandes rasgos, este libro desarrolla el camino que emprende un grupo de aventureros en un entorno similar a la mitologia griega (con dioses e historia específicos del setting del libro) para salvar a su mundo de la perdición, y cómo ellos logran ascender a la divinidad en el proceso.

Por cuestiones de tiempos de entrenamiento y para facilitar probar distintos tipos de modelo, se redujo a los primeros dos capitulos (80 paginas) de este libro.

In [2]:
with open("Odyssey of the Dragonlords.txt", "r") as file:
    lines = file.readlines()
    article_text = " ".join(line.strip() for line in lines)

# en article text se encuentra el texto de todo el libro
article_text = article_text.lower()

### Elegir el tamaño del contexto

En este caso, como el modelo de lenguaje es por caracteres, todo un gran corpus
de texto puede ser considerado un documento en sí mismo y el tamaño de contexto
puede ser elegido con más libertad en comparación a un modelo de lenguaje tokenizado por palabras y dividido en documentos más acotados.

In [3]:
# seleccionamos el tamaño de contexto
max_context_size = 100

# en este caso el vocabulario es el conjunto único de caracteres que existe en todo el texto
chars_vocab = set(article_text)

# la longitud de vocabulario de caracteres es:
print(f"Vocabulario: {len(chars_vocab)} letras")

# Construimos los dicionarios que asignan índices a caracteres y viceversa.
# El diccionario `char2idx` servirá como tokenizador.
char2idx = {k: v for v,k in enumerate(chars_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

Vocabulario: 44 letras


###  Tokenizar

In [4]:
# tokenizamos el texto completo
tokenized_text = [char2idx[ch] for ch in article_text]

### Organizando y estructurando el dataset

In [5]:
# separaremos el dataset entre entrenamiento y validación.
# `p_val` será la proporción del corpus que se reservará para validación
# `num_val` es la cantidad de secuencias de tamaño `max_context_size` que se usará en validación
p_val = 0.1
num_val = int(np.ceil(len(tokenized_text)*p_val/max_context_size))

# separamos la porción de texto utilizada en entrenamiento de la de validación.
train_text = tokenized_text[:-num_val*max_context_size]
val_text = tokenized_text[-num_val*max_context_size:]

tokenized_sentences_train = [train_text[init:init+max_context_size] for init in range(len(train_text)-max_context_size+1)]
tokenized_sentences_val = [val_text[init*max_context_size:init*(max_context_size+1)] for init in range(num_val)]

X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])

vocab_size = len(chars_vocab)

Nótese que estamos estructurando el problema de aprendizaje como *many-to-many*:

Entrada: secuencia de tokens [$x_0$, $x_1$, ..., $x_N$]

Target: secuencia de tokens [$x_1$, $x_2$, ..., $x_{N+1}$]

De manera que la red tiene que aprender que su salida deben ser los tokens desplazados en una posición y un nuevo token predicho (el N+1).

La ventaja de estructurar el aprendizaje de esta manera es que para cada token de target se propaga una señal de gradiente por el grafo de cómputo recurrente, que es mejor que estructurar el problema como *many-to-one* en donde sólo una señal de gradiente se propaga.

En este punto tenemos en la variable `tokenized_sentences` los versos tokenizados. Vamos a quedarnos con un conjunto de validación que utilizaremos para medir la calidad de la generación de secuencias con la métrica de Perplejidad.

# Definiendo el modelo

El ejemplo dado estaba implementado con Keras y TensorFlow, pero a partir de la version 2.10 ya no puede utilizar GPU en Windows nativamente. Intenté probar instalando versiones anteriores y WSL (windows subsystem for linux) pero ambas opciones me estaban dando problemas (que de hecho fue una de las causas por las que estoy entregando esta tarea mas tarde de lo deseado). Asi que decidí implementarlo en PyTorch.

In [6]:
class SequenceDataset(Dataset):
    def __init__(self, sequences, max_context_size):
        self.sequences = sequences
        self.max_context_size = max_context_size
        self.inputs, self.targets = self.prepare_sequences()

    def prepare_sequences(self):
        inputs = []
        targets = []
        for seq in self.sequences:
            len_seq = len(seq)
            subseq = [seq[:i] for i in range(1, len_seq)]
            if len(subseq) > 0:
                padded_subseq = pad_sequence([torch.tensor(s) for s in subseq], batch_first=True, padding_value=0)
                inputs.extend(padded_subseq)
                targets.extend([seq[i] for i in range(1, len_seq)])
        return inputs, torch.tensor(targets)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [7]:
class SimpleRNNModel(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super(SimpleRNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, vocab_size)
        self.rnn = nn.RNN(vocab_size, hidden_size, batch_first=True, dropout=0.1)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = self.fc(x)
        x = self.softmax(x)
        return x


In [8]:
def collate_fn(batch):
    inputs = [item[0] for item in batch]
    targets = torch.tensor([item[1] for item in batch])
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)
    return inputs_padded, targets

In [10]:
def train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, patience, device):
    history_ppl = []
    min_score = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs[:, -1, :], targets)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        scores = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                probs = [outputs[i, -1, target].item() for i, target in enumerate(targets)]
                scores.append(np.exp(-np.sum(np.log(probs)) / len(probs)))

        current_score = np.mean(scores)
        history_ppl.append(current_score)
        print(f'\n mean perplexity: {current_score} \n')

        # Early stopping
        if current_score < min_score:
            min_score = current_score
            torch.save(model.state_dict(), "my_model.pth")
            print("Saved new model!")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter == patience:
                print("Stopping training...")
                break

In [11]:
emb_size = 50
hidden_size = 100
batch_size = 4096
learning_rate = 1e-3
num_epochs = 20
patience = 5

In [124]:
train_dataset = SequenceDataset(tokenized_sentences_train, max_context_size)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [125]:
val_dataset = SequenceDataset(tokenized_sentences_val, max_context_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleRNNModel(vocab_size, emb_size, hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

c:\Users\Nicolas\anaconda3\envs\dataAnalysis\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [132]:
train_model(model, train_loader, val_loader, num_epochs, criterion, optimizer, patience, device)


 mean perplexity: 20694.260532839002 

Saved new model!

 mean perplexity: 105994.23841181268 


 mean perplexity: 416476.1314922673 


 mean perplexity: 1584761.2549259802 


 mean perplexity: 5081010.936265317 



KeyboardInterrupt: 

In [77]:
model = Sequential()

model.add(TimeDistributed(CategoryEncoding(num_tokens=vocab_size, output_mode = "one_hot"),input_shape=(None,1)))
model.add(SimpleRNN(200, return_sequences=True, dropout=0.1, recurrent_dropout=0.1 ))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_3 (TimeDi  (None, None, 46)          0         
 stributed)                                                      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, None, 200)         49400     
                                                                 
 dense_3 (Dense)             (None, None, 46)          9246      
                                                                 
Total params: 58646 (229.09 KB)
Trainable params: 58646 (229.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________



### Definir el modelo

Dado que por el momento no hay implementaciones adecuadas de la perplejidad que puedan operar en tiempo de entrenamiento, armaremos un Callback *ad-hoc* que la calcule en cada epoch.

**Nota**: un Callback es una rutina gatillada por algún evento, son muy útiles para relevar datos en diferentes momentos del desarrollo del modelo. En este caso queremos hacer un cálculo cada vez que termina una epoch de entrenamiento.

In [78]:
class PplCallback(keras.callbacks.Callback):

    '''
    Este callback es una solución ad-hoc para calcular al final de cada epoch de
    entrenamiento la métrica de Perplejidad sobre un conjunto de datos de validación.
    La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias.
    Además implementa la finalización del entrenamiento (Early Stopping)
    si la perplejidad no mejora después de `patience` epochs.
    '''

    def __init__(self, val_data, history_ppl,patience=5):
      # El callback lo inicializamos con secuencias de validación sobre las cuales
      # mediremos la perplejidad
      self.val_data = val_data

      self.target = []
      self.padded = []

      count = 0
      self.info = []
      self.min_score = np.inf
      self.patience_counter = 0
      self.patience = patience

      # nos movemos en todas las secuencias de los datos de validación
      for seq in self.val_data:

        len_seq = len(seq)
        # armamos todas las subsecuencias
        subseq = [seq[:i] for i in range(1,len_seq)]
        self.target.extend([seq[i] for i in range(1,len_seq)])

        if len(subseq)!=0:

          self.padded.append(pad_sequences(subseq, maxlen=max_context_size, padding='pre'))

          self.info.append((count,count+len_seq))
          count += len_seq

      self.padded = np.vstack(self.padded)


    def on_epoch_end(self, epoch, logs=None):

        # en `scores` iremos guardando la perplejidad de cada secuencia
        scores = []

        predictions = self.model.predict(self.padded,verbose=0)

        # para cada secuencia de validación
        for start,end in self.info:

          # en `probs` iremos guardando las probabilidades de los términos target
          probs = [predictions[idx_seq,-1,idx_vocab] for idx_seq, idx_vocab in zip(range(start,end),self.target[start:end])]

          # calculamos la perplejidad por medio de logaritmos
          scores.append(np.exp(-np.sum(np.log(probs))/(end-start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores)
        history_ppl.append(current_score)
        print(f'\n mean perplexity: {current_score} \n')

        # chequeamos si tenemos que detener el entrenamiento
        if current_score < self.min_score:
          self.min_score = current_score
          self.model.save("my_model.keras")
          print("Saved new model!")
          self.patience_counter = 0
        else:
          self.patience_counter += 1
          if self.patience_counter == self.patience:
            print("Stopping training...")
            self.model.stop_training = True


### Entrenamiento

In [80]:
# fiteamos, nótese el agregado del callback con su inicialización. El batch_size lo podemos seleccionar a mano
# en general, lo mejor es escoger el batch más grande posible que minimice el tiempo de cada época.
# En la variable `history_ppl` se guardarán los valores de perplejidad para cada época.
history_ppl = []
hist = model.fit(X, y, epochs=5, callbacks=[PplCallback(tokenized_sentences_val,history_ppl)], batch_size=1024)

Epoch 1/5
158/158 [==============================] - ETA: 0s - loss: 2.6069
 mean perplexity: 10.830139672811628 

Saved new model!
158/158 [==============================] - 73s 463ms/step - loss: 2.6069
Epoch 2/5
158/158 [==============================] - ETA: 0s - loss: 2.2887
 mean perplexity: 9.410327607071721 

Saved new model!
158/158 [==============================] - 73s 462ms/step - loss: 2.2887
Epoch 3/5
 73/158 [============>.................] - ETA: 36s - loss: 2.2144

KeyboardInterrupt: 

In [160]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(history_ppl) + 1)
sns.lineplot(x=epoch_count,  y=history_ppl)
plt.show()

NameError: name 'history_ppl' is not defined

In [14]:
# Cargamos el mejor modelo guardado del entrenamiento para hacer inferencia
model = model.load_state_dict(torch.load('my_model.pth'))


### Predicción del próximo caracter

In [ ]:
# Se puede usar gradio para probar el modelo
# Gradio es una herramienta muy útil para crear interfaces para ensayar modelos
# https://gradio.app/

!pip install -q gradio

In [25]:
# Function to make a prediction
def model_response(human_text):
    # Encode the input text
    encoded = [char2idx[ch] for ch in human_text.lower() if ch in char2idx]
    
    # Convert to tensor and add batch dimension
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)  # Shape: (1, sequence_length)
    
    # Pad the sequence if necessary
    if encoded_tensor.size(1) < max_context_size:
        padding = torch.zeros((1, max_context_size - encoded_tensor.size(1)), dtype=torch.long)
        encoded_tensor = torch.cat((padding, encoded_tensor), dim=1)
    else:
        encoded_tensor = encoded_tensor[:, -max_context_size:]  # Truncate if necessary

    # Make a prediction
    with torch.no_grad():
        output = model(encoded_tensor)
    
    # Get the predicted token
    y_hat = torch.argmax(output[0, -1, :]).item()
    
    # Map the predicted index back to a character
    out_word = idx2char[y_hat]
    
    # Append the predicted character to the input text
    return human_text + out_word

def model_responses(human_text):
    return
    # Encodeamos
    encoded = [char2idx[ch] for ch in human_text.lower() ]
    # Si tienen distinto largo
    encoded = pad_sequences([encoded], maxlen=max_context_size, padding='pre')

    # Predicción softmax
    y_hat = np.argmax(model.predict(encoded)[0,-1,:])


    # Debemos buscar en el vocabulario el caracter
    # que corresopnde al indice (y_hat) predicho por le modelo
    out_word = ''
    out_word = idx2char[y_hat]

    # Agrego la palabra a la frase predicha
    return human_text + out_word

iface = gr.Interface(
    fn=model_response,
    inputs=["textbox"],
    outputs="text")

iface.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Nicolas\anaconda3\envs\dataAnalysis\Lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nicolas\anaconda3\envs\dataAnalysis\Lib\site-packages\gradio\route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nicolas\anaconda3\envs\dataAnalysis\Lib\site-packages\gradio\blocks.py", line 1897, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nicolas\anaconda3\envs\dataAnalysis\Lib\site-packages\gradio\blocks.py", line 1483, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nicolas\anaconda3\envs\dataAnalysis\Lib\site-packages\anyio\to_thread.py", line 56, in run_sy

Keyboard interruption in main thread... closing server.


### Generación de secuencias

In [30]:
def generate_seq(model, seed_text, max_length, n_words):
    """
        Exec model sequence prediction

        Args:
            model (keras): modelo entrenado
            seed_text (string): texto de entrada (input_seq)
            max_length (int): máxima longitud de la sequencia de entrada
            n_words (int): números de caracteres a agregar a la sequencia de entrada
        returns:
            output_text (string): sentencia con las "n_words" agregadas
    """
    output_text = seed_text
	# generate a fixed number of words
    for _ in range(n_words):
		# Encodeamos
        encoded = [char2idx[ch] for ch in output_text.lower() ]
		# Si tienen distinto largo
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

		# Predicción softmax
        y_hat = np.argmax(model.predict(encoded,verbose=0)[0,-1,:])
		# Vamos concatenando las predicciones
        out_word = ''

        out_word = idx2char[y_hat]

		# Agrego las palabras a la frase predicha
        output_text += out_word
    return output_text

In [31]:
input_text='habia una vez'

generate_seq(model, input_text, max_length=max_context_size, n_words=30)

'habia una vez en la carra de la compañeros '

###  Beam search y muestreo aleatorio

In [32]:
# funcionalidades para hacer encoding y decoding

def encode(text,max_length=max_context_size):

    encoded = [char2idx[ch] for ch in text]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')

    return encoded

def decode(seq):
    return ''.join([idx2char[ch] for ch in seq])

In [33]:
from scipy.special import softmax

# función que selecciona candidatos para el beam search
def select_candidates(pred,num_beams,vocab_size,history_probs,history_tokens,temp,mode):

  # colectar todas las probabilidades para la siguiente búsqueda
  pred_large = []

  for idx,pp in enumerate(pred):
    pred_large.extend(np.log(pp+1E-10)+history_probs[idx])

  pred_large = np.array(pred_large)

  # criterio de selección
  if mode == 'det':
    idx_select = np.argsort(pred_large)[::-1][:num_beams] # beam search determinista
  elif mode == 'sto':
    idx_select = np.random.choice(np.arange(pred_large.shape[0]), num_beams, p=softmax(pred_large/temp)) # beam search con muestreo aleatorio
  else:
    raise ValueError(f'Wrong selection mode. {mode} was given. det and sto are supported.')

  # traducir a índices de token en el vocabulario
  new_history_tokens = np.concatenate((np.array(history_tokens)[idx_select//vocab_size],
                        np.array([idx_select%vocab_size]).T),
                      axis=1)

  # devolver el producto de las probabilidades (log) y la secuencia de tokens seleccionados
  return pred_large[idx_select.astype(int)], new_history_tokens.astype(int)


def beam_search(model,num_beams,num_words,input,temp=1,mode='det'):

    # first iteration

    # encode
    encoded = encode(input)

    # first prediction
    y_hat = model.predict(encoded,verbose=0)[0,-1,:]

    # get vocabulary size
    vocab_size = y_hat.shape[0]

    # initialize history
    history_probs = [0]*num_beams
    history_tokens = [encoded[0]]*num_beams

    # select num_beams candidates
    history_probs, history_tokens = select_candidates([y_hat],
                                        num_beams,
                                        vocab_size,
                                        history_probs,
                                        history_tokens,
                                        temp,
                                        mode)

    # beam search loop
    for i in range(num_words-1):

      preds = []

      for hist in history_tokens:

        # actualizar secuencia de tokens
        input_update = np.array([hist[i+1:]]).copy()

        # predicción
        y_hat = model.predict(input_update,verbose=0)[0,-1,:]

        preds.append(y_hat)

      history_probs, history_tokens = select_candidates(preds,
                                                        num_beams,
                                                        vocab_size,
                                                        history_probs,
                                                        history_tokens,
                                                        temp,
                                                        mode)

    return history_tokens[:,-(len(input)+num_words):]

In [34]:
# predicción con beam search
salidas = beam_search(model,num_beams=10,num_words=20,input="habia una vez")

In [35]:
salidas[0]

array([30, 17, 50,  3, 17, 56, 65,  9, 17, 56, 26, 22, 46, 56, 29,  3, 28,
        5, 22, 41, 56, 38, 47, 58, 58, 56, 28, 22, 56, 30, 17, 50, 57])

In [36]:
# veamos las salidas
decode(salidas[0])

'habia una vez mister fogg se habí'